# Stage 1: Generate Stock Universe

- Gather stocks of interest
- Gather stocks from specific criteria (SP500 top 50...)
- Gather stocks from specific portfolio account
- Assemble stock universe 
- Use stock sentiment to select stocks
- Gather price histories

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

## Authentication Tokens

To get data from Ameritrade you will need to obtains a short time use token (there is a re-use token, but I have not coded it yet.) You only need to do this if you
are going to use an existing Ameritrade account to define an initial set of stocks to analyze.

To obtain a token, you will need to have a Chrome driver located somewhere on your system. This will allow the module to use your credentials to obtain an authentication token.

For security reasons, I sugges using environment variables to store your credential information. If you store them in property files, or just code them into your notebook, you risk sharing the information with others if you use GitHub or some other SCCS. This also makes it easier to have them availabe from project to project in your development environment

<span style="color:blue">Note: *Account numbers are masked for security purposes.*</span>

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

# For Chromedriver
from pathlib import Path
chrome_executabel_path = str(Path.home()) + r'\Anaconda Projects\chromedriver\chromedriver'

# Make sure we have a data directory
Path('./data').mkdir(parents=True, exist_ok=True) 

# Which account are we interested in
masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

In [3]:
snp_500_df = utils.get_snp500()
snp_500_symbols = snp_500_df.index.to_list()
quote_dfs = []
for i in range(0, 500, 100):
    quote_dfs.append(amc.AmeritradeRest(username, password, client_id).get_quotes(snp_500_symbols[i:i+100]))
    
snp_500_quotes_df = pd.concat(quote_dfs, axis=0)
snp_500_quotes_df.describe()

snp_500_tickers = set(snp_500_df.index.to_list())
print(len(snp_500_tickers))

505


In [4]:
dow_tikers = set(utils.get_dow().index.to_list())
print(len(dow_tikers))

30


## Get sentiment data from Finvis

In [5]:
stock_universe = set.union(snp_500_tickers, dow_tikers)
print(len(stock_universe))

505


In [6]:
parsed_and_scored_news = utils.get_finvis_stock_sentiment(snp_500_symbols).sort_values(by='date')
parsed_and_scored_news

News Tables: 100%|████████████████████████████████████████████████████| 503/503 [00:02<00:00, 214.43News Table Items/s]


,ticker,date,time,headline,neg,neu,pos,compound
29206,L,2019-10-28,06:00AM,Loews Corporation Reports Net Income Of $72 Mi...,0.000,1.000,0.000,0.0000
29205,L,2019-10-28,06:11AM,Loews reports slump in third-quarter profit as...,0.000,0.775,0.225,0.4404
29204,L,2019-10-28,09:00AM,Loews Corp. to Host Earnings Call,0.000,1.000,0.000,0.0000
29203,L,2019-10-28,12:19PM,"Loews (L) Q3 Earnings Miss Estimates, Revenues...",0.167,0.833,0.000,-0.1531
29202,L,2019-10-28,03:26PM,Global Markets Advance Monday,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...
18907,FDX,2021-09-30,06:54AM,"Zacks Earnings Trends Highlights: JPMorgan, Or...",0.000,1.000,0.000,0.0000
4408,AAPL,2021-09-30,03:16AM,Lordstown Jumps on Report Foxconn May Buy its ...,0.000,1.000,0.000,0.0000
4407,AAPL,2021-09-30,04:06AM,Security experts urge iPhone users to remove V...,0.089,0.759,0.152,0.2500
48223,WFC,2021-09-30,06:00AM,Influencers with Andy Serwer: Suzanne Shank,0.000,1.000,0.000,0.0000


In [7]:
# Group by date and ticker columns from scored_news and calculate the mean
mean_scores = parsed_and_scored_news.groupby(['ticker','date']).mean().fillna(0)
# Unstack the column ticker
mean_scores = mean_scores.unstack()
# Get the cross-section of compound in the 'columns' axis
mean_scores = mean_scores.xs('compound', axis="columns").transpose().fillna(0)
# Get cusmum score of each stock
cum_scores = mean_scores[-40:].cumsum(axis=0)
current_scores = cum_scores.iloc[-1]
mean_score = current_scores.mean()
stdv_score = current_scores.std()
cutoff = mean_score - stdv_score

print(mean_score, stdv_score, cutoff)

2.2365715542510105 1.7674201192322272 0.4691514350187833


In [8]:
stock_universe = current_scores.where(current_scores > cutoff).dropna().index.to_list()

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

We will obtain 5 years of price histories. In the end this will provide us with 2 years of factor data since some of the factors are based on 1 year returns.

In [9]:
number_of_years = 5
price_histories = amc.AmeritradeRest(username, password, client_id).get_price_histories(stock_universe, datetime.today().strftime('%Y-%m-%d'), num_periods=number_of_years)
utils.save_price_histories(price_histories, price_histories_file_name)

Tickers:   0%|          | 0/437 [00:00<?, ?Price Histories/s]

In [10]:
price_histories.head()

,open,high,low,close,volume,ticker,date
0,46.500000,47.315000,46.300000,47.090000,1754316,A,2016-09-30
64792,306.590000,314.380000,306.040000,313.030000,2124784,BIIB,2016-09-30
434996,12.893333,13.044444,12.813333,13.013333,1217412,ROL,2016-09-30
66050,162.610000,164.450000,161.510000,163.810000,181747,BIO,2016-09-30
433737,119.500000,123.110000,119.270000,122.340000,2568222,ROK,2016-09-30


In [11]:
price_histories = utils.read_price_histories(price_histories_file_name)
close = utils.get_close_values(price_histories)
close.tail()

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2021-09-24 00:00:00+00:00,172.400,21.3900,215.66,146.92,107.070,122.30,124.37,339.23,622.71,178.53,...,82.88,63.32,57.59,60.01,135.62,125.520,151.23,559.480,60.25,204.73
2021-09-27 00:00:00+00:00,166.380,21.4700,218.46,145.37,107.720,122.63,120.58,334.51,603.65,176.91,...,85.38,62.25,59.30,60.73,134.30,125.570,151.54,548.580,62.89,197.80
2021-09-28 00:00:00+00:00,159.900,21.4300,214.78,141.91,107.340,121.29,118.74,324.21,578.77,171.29,...,84.31,61.42,59.92,59.14,128.92,122.540,150.85,531.180,62.74,192.77
2021-09-29 00:00:00+00:00,160.000,21.0100,216.77,142.83,108.840,122.50,119.46,323.53,577.70,168.84,...,83.63,62.67,59.88,58.43,127.78,123.040,149.36,525.670,63.74,194.99
2021-09-30 00:00:00+00:00,161.725,20.5447,213.07,142.92,110.025,122.41,120.49,323.53,582.69,170.31,...,84.02,62.75,59.34,58.67,126.95,123.135,148.82,523.635,63.36,198.12
